# 표11-1. 수요는 변동하지만 공급 리드타임은 일정한 경우, ChatGPT 요청

물류센터의 최근 3개월간의 TV 출고정보는 GI.xlsx 저장되어 있다. 공장에서 물류센터로 리드타임은 4일로 일정하다. 서비스 수준이 99%(결품 허용확률 1%)일 때, 안전재고와 발주점은 얼마인가?

# 도표 11-5. 공급이 일정한 경우의 안전재고 계산 파이썬 코드
-	데이터 읽어 오기

In [3]:
# pandas 라이브러리를 임포트하여 데이터 조작 및 분석 기능을 사용할 수 있도록 함
import pandas as pd

# pd.read_excel 함수를 사용하여 지정된 경로의 Excel 파일을 로드
# 'data'라는 변수에 로드된 데이터를 DataFrame 형태로 저장
data = pd.read_excel("/content/sample_data/GI.xlsx")

# DataFrame의 head 메소드를 사용하여 로드된 데이터의 처음 5행을 출력
# 이를 통해 데이터의 구조와 함께 주요 내용을 빠르게 확인할 수 있음
data.head()

,DAY,GI
0,2023-07-01,58820
1,2023-07-02,52000
2,2023-07-03,54893
3,2023-07-04,61204
4,2023-07-05,59337


# 도표 11-5. 공급이 일정한 경우의 안전재고 계산 파이썬 코드

-	수요(출고량)의 평균과 표준편차 출력

In [4]:
# pandas 라이브러리를 사용하여 'data'라는 DataFrame을 이용해 일일 출고량 데이터를 분석

# 'GI' 열에서 일일 출고량의 평균을 계산하고 'mean_daily_gi'라는 변수에 저장
mean_daily_gi = data['GI'].mean()

# 'GI' 열에서 일일 출고량의 표준편차를 계산하고 'std_daily_gi'라는 변수에 저장
std_daily_gi = data['GI'].std()

# 계산된 평균과 표준편차를 출력
mean_daily_gi, std_daily_gi


(50076.032608695656, 5160.248340243997)

In [5]:
# scipy의 stats 모듈에서 norm을 임포트하여 정규 분포 관련 함수를 사용할 수 있게 함

from scipy.stats import norm

# norm.ppf 함수를 사용하여 서비스 수준 99%에 해당하는 Z 값을 계산
# ppf는 percent point function (분위수 함수)로, 주어진 확률에 대한 분포의 값(Z 값)을 반환
z_value = norm.ppf(0.99)

# 안전재고를 계산
# 안전재고는 예측 불가능한 수요 증가를 대비하여 보유해야 하는 추가 재고량
# 이는 Z 값, 일일 출고량의 표준편차, 그리고 리드타임의 제곱근을 곱하여 계산
safety_stock = z_value * std_daily_gi * (4**0.5)

# 발주점을 계산
# 발주점은 총 수요가 예상되는 지점에서 주문을 해야 하는 재고 수준
# 일일 평균 수요에 리드타임을 곱한 값에 안전재고를 더해 계산
reorder_point = mean_daily_gi * 4 + safety_stock

# 계산된 Z 값, 안전재고, 발주점을 출력
z_value, safety_stock, reorder_point

(2.3263478740408408, 24009.0655116988, 224313.19594648143)

# 표 11-6. 발주점이 주어진 상태에서 ChatGPT의 서비스수준 확인

물류센터의 최근 3개월간의 TV 출고정보는 GI.xlsx 저장되어 있다. 공장에서 물류센터로 리드타임은 4일로 일정하다. 발주점을 215,000으로 설정하였다면, 서비스 수준은 얼마인가?

In [6]:
# 데이터의 'GI' 열에 대한 기술적 통계를 계산합니다.
summary_statistics = data['GI'].describe()
# 데이터셋의 총 날짜 수를 계산합니다. 이는 유니크한 'DAY' 값의 개수입니다.
total_days = data['DAY'].nunique()
# 기술적 통계와 총 날짜 수를 출력합니다.
summary_statistics, total_days

(count       92.000000
 mean     50076.032609
 std       5160.248340
 min      37235.000000
 25%      46503.500000
 50%      50243.500000
 75%      53685.000000
 max      61348.000000
 Name: GI, dtype: float64,
 92)

In [7]:
# 평균 일일 출고량을 계산합니다. 이는 'GI' 열의 평균값을 사용합니다.
average_daily_demand = summary_statistics['mean']

# 리드타임을 일수로 설정합니다. 여기서는 4일입니다.
lead_time_days = 4

# 리드타임 동안의 총 예상 출고량을 계산합니다. 이는 평균 일일 출고량에 리드타임 일수를 곱한 값입니다.
lead_time_demand = average_daily_demand * lead_time_days

# 발주점에서 리드타임 동안의 총 출고량을 뺀 값으로 안전 재고를 계산합니다.
safety_stock = 215000 - lead_time_demand

# 정규분포를 사용하여 서비스 수준을 추정하기 위한 Z-점수를 계산합니다.
from scipy.stats import norm

# 일일 출고량의 표준편차를 사용합니다.
std_dev = summary_statistics['std']

# 안전 재고와 리드타임 동안의 표준편차를 사용하여 Z-점수를 계산합니다.
z_score = safety_stock / (std_dev * (lead_time_days ** 0.5))

# Z-점수에 해당하는 서비스 수준을 계산합니다. 이는 정규분포의 누적분포함수(CDF)를 사용하여 Z-점수 이하의 확률을 나타냅니다.
service_level = norm.cdf(z_score)

# 리드타임 동안의 예상 출고량, 안전 재고, Z-점수, 서비스 수준을 출력합니다.
lead_time_demand, safety_stock, z_score, service_level

(200304.13043478262,
 14695.869565217377,
 1.4239498369299897,
 0.9227695031779872)

# 도표 11-10. 수요는 일정, 리드타임이 변동하는 경우의 ChatGPT 안전재고 계산

TV를 보관하는 물류센터에서 하루 평균 20개의 TV를 출하하고 있다. 공장에서 물류센터로 공급 리드타임의 과거 3개월간 실적 정보는 SupplyLT.xlsx에 저장되어 있다. 허용 품절율이 3%일 때, 안전재고와 발주점은 얼마인가?

# 도표 11-10. 수요는 일정, 리드타임이 변동하는 경우의 ChatGPT 안전재고 계산

In [8]:
import pandas as pd

# 데이터 파일 로드
data = pd.read_excel('/content/sample_data/SupplyLT.xlsx')

# 데이터 통계 확인
data.describe()

,LT
count,92.000000
mean,6.054348
std,2.066819
min,1.000000
25%,5.000000
50%,6.000000
75%,7.250000
max,11.000000


In [9]:
from scipy.stats import norm

# 서비스 수준에 따른 z 값 계산
z_value = norm.ppf(0.97)

# 안전재고 계산
mu_L = 6.05
sigma_D = 0  # 일정한 수요로 인해 표준편차는 0
mu_D = 20
sigma_L = 2.07

safety_stock = z_value * ((mu_L * sigma_D**2) + (mu_D**2 * sigma_L**2))**0.5

# 발주점(Reorder Point) 계산
reorder_point = mu_L * mu_D + safety_stock

safety_stock, reorder_point


(77.86485537746178, 198.86485537746177)

# 도표 10-11. 안전재고 계산 공식을 지정한 경우의 요청과 답변

TV를 보관하는 물류센터에서 과거 3개월의 TV 출하실적정보는 GI.xlsx 저장되어 있다. 공장에서 물류센터로 리드타임의 과거 3개월 실적 정보는 SupplyLT.xlsx에 저장되어 있다. 서비스 수준을 90%로 유지하고자 할 때, 안전재고와 발주점은 얼마인가?
- 안전 재고는 (일일 수요의 표준편차의 제곱 × 리드타임의 평균)과 (일일 수요 평균의 제곱 × 리드타임의 표준편차의 제곱)을 더한 후 제곱근을 하고, 안전계수와 곱해서 계산한다

In [15]:
# Load the new data from the updated Excel files
gi_df_new = pd.read_excel('/content/sample_data/GI.xlsx')
supplylt_df_new = pd.read_excel('/content/sample_data/SupplyLT.xlsx')

# Calculate the mean and standard deviation for daily shipments (demand)
daily_demand_mean_new = gi_df_new['GI'].mean()
daily_demand_std_new = gi_df_new['GI'].std()

# Calculate the mean and standard deviation for lead time
lead_time_mean_new = supplylt_df_new['LT'].mean()
lead_time_std_new = supplylt_df_new['LT'].std()

daily_demand_mean_new, daily_demand_std_new, lead_time_mean_new, lead_time_std_new


(50076.032608695656, 5160.248340243997, 6.054347826086956, 2.066818601240753)

In [24]:
import numpy as np
# Z-score for 90% service level
z_score = 1.28
# z_score = norm.ppf(0.90)

# Calculate Safety Stock using the provided formula
safety_stock_new = z_score * np.sqrt((daily_demand_std_new ** 2 * lead_time_mean_new) + (daily_demand_mean_new ** 2 * lead_time_std_new ** 2))

# Calculate Reorder Point (ROP)
reorder_point_new = (daily_demand_mean_new * lead_time_mean_new) + safety_stock_new

safety_stock_new, reorder_point_new


(133632.51256777626, 436810.2317312923)

In [23]:
z_score

1.2815515655446004